In [4]:
import os

os.environ["GROQ_API_KEY"] = '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'#getpass.getpass("Enter API key for Groq: ")
os.environ["TAVILY_API_KEY"] = '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'

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743846987, 'localtime': '2025-04-05 02:56'}, 'current': {'last_updated_epoch': 1743846300, 'last_updated': '2025-04-05 02:45', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 275, 'wind_dir': 'W', 'pressure_mb': 1024.0, 'pressure_in': 30.25, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 50, 'feelslike_c': 9.5, 'feelslike_f': 49.2, 'windchill_c': 8.5, 'windchill_f': 47.3, 'heatindex_c': 8.9, 'heatindex_f': 48.1, 'dewpoint_c': 8.4, 'dewpoint_f': 47.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 7.7, 'gust_kph': 12.5}}", 'score': 0.9

In [28]:
#from langchain.chat_models import init_chat_model
#model = init_chat_model("llama3-8b-8192", model_provider="groq")
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")  # Replace with actual model if different


In [29]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [30]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi!'}, 'id': 'call_04a4', 'type': 'tool_call'}]


In [32]:
# response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

# print(f"ContentString: {response.content}")
# print(f"ToolCalls: {response.tool_calls}")

In [33]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [34]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='cd76f422-2dad-482c-874d-596e0f8e7d00'),
 AIMessage(content='Hello!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 1903, 'total_tokens': 1906, 'completion_time': 0.0025, 'prompt_time': 0.237365827, 'queue_time': -0.292785408, 'total_time': 0.239865827}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-98476141-226b-4946-8d41-33b6b81d1038-0', usage_metadata={'input_tokens': 1903, 'output_tokens': 3, 'total_tokens': 1906})]

In [35]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what is temprature right now?")]}
)
response["messages"]

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_calls": [\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "current_weather"\n\t\t\t},\n\t\t\t"parameters": {\n\t\t\t\t"location": "your current location"\n\t\t\t}\n\t\t}\n\t]\n}\n</tool-use>'}}

In [37]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [38]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [39]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1906, 'total_tokens': 1910, 'completion_time': 0.003333333, 'prompt_time': 0.236235731, 'queue_time': -0.298196441, 'total_time': 0.239569064}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-aaa19d4e-3586-47df-870c-181deca7d415-0', usage_metadata={'input_tokens': 1906, 'output_tokens': 4, 'total_tokens': 1910})]}}
----


In [40]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 961, 'total_tokens': 967, 'completion_time': 0.005, 'prompt_time': 0.119937242, 'queue_time': 0.019847433999999997, 'total_time': 0.124937242}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-5349b408-fe92-409c-a95c-b89d7a8d5aec-0', usage_metadata={'input_tokens': 961, 'output_tokens': 6, 'total_tokens': 967})]}}
----


In [41]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm not sure what your name is. I'm a large language model, I don't have access to any information about your personal identity. Can you please provide me with more context or clarify your question?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 1909, 'total_tokens': 1952, 'completion_time': 0.035833333, 'prompt_time': 0.253438217, 'queue_time': -0.30243614900000004, 'total_time': 0.28927155}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-690bb5a0-cb9c-4ecb-81da-7ce42c24a5a9-0', usage_metadata={'input_tokens': 1909, 'output_tokens': 43, 'total_tokens': 1952})]}}
----
